In [432]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/noshowappointments/KaggleV2-May-2016.csv


[참고 링크1](https://www.kaggle.com/code/somrikbanerjee/predicting-show-up-no-show)  
[참고 링크2](https://www.kaggle.com/code/tsilveira/applying-heatmaps-for-categorical-data-analysis)  
[참고 링크3](https://www.kaggle.com/code/ahmedmohameddawoud/why-patients-do-not-show-up-for-the-appointment)  
[참고 링크4](https://www.kaggle.com/code/abdelnaemalaref/medical-appointment-no-show)



In [433]:
import seaborn as sns
sns.set_style("whitegrid")

data = pd.read_csv('/kaggle/input/noshowappointments/KaggleV2-May-2016.csv')
print(data.head())

# Hipertension 고혈압
# Diabetes 당뇨병
# Alcoholism 알콜중독
# Handcap 장애

      PatientId  AppointmentID Gender          ScheduledDay  \
0  2.987250e+13        5642903      F  2016-04-29T18:38:08Z   
1  5.589978e+14        5642503      M  2016-04-29T16:08:27Z   
2  4.262962e+12        5642549      F  2016-04-29T16:19:04Z   
3  8.679512e+11        5642828      F  2016-04-29T17:29:31Z   
4  8.841186e+12        5642494      F  2016-04-29T16:07:23Z   

         AppointmentDay  Age      Neighbourhood  Scholarship  Hipertension  \
0  2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0             1   
1  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             0   
2  2016-04-29T00:00:00Z   62      MATA DA PRAIA            0             0   
3  2016-04-29T00:00:00Z    8  PONTAL DE CAMBURI            0             0   
4  2016-04-29T00:00:00Z   56    JARDIM DA PENHA            0             1   

   Diabetes  Alcoholism  Handcap  SMS_received No-show  
0         0           0        0             0      No  
1         0           0        0      

## 데이터 확인 및 정리

In [434]:
data.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [435]:
# Age -1 제거
data.drop(data.query("Age == -1").index,inplace=True)

In [436]:
data["No-show"].value_counts()

No     88207
Yes    22319
Name: No-show, dtype: int64

## 새로운 Feature 추가 1
[참고 자료](https://www.kaggle.com/code/orimeidler/no-show-report?scriptVersionId=89034999&cellId=5)

In [437]:
data["ScheduledHour"] = pd.to_datetime(data["ScheduledDay"]).apply(lambda p:p.hour)

## 새로운 Feature 추가 2
[참고자료](https://www.kaggle.com/code/tsilveira/applying-heatmaps-for-categorical-data-analysis?scriptVersionId=2688863&cellId=45)

In [438]:
## Converting the date information in string to datetime type:
data['ScheduledDay'] = pd.to_datetime(data.ScheduledDay)
data['AppointmentDay'] = pd.to_datetime(data.AppointmentDay)
## Creating a new column (attribute) containing just the scheduling time:
data['ScheduleTime'] = data.ScheduledDay.dt.time
## Normalizing the "Day" columns to keep just the date information (dropping the time info)
data['ScheduledDay'] = data.ScheduledDay.dt.normalize()

## Since both 'AppointmentDay' and 'ScheduledDay' are pandas.Timestamp type, this operation can be done directly:
data['WaitingDays'] = data['AppointmentDay'] - data['ScheduledDay']

def waiting_days(days):
    '''Auxiliary function to parse a date information from string type to python datetime object.
    Syntax: waiting_days(days), where:
        days = int type with the number of days considered.
    Return: a correspondent pandas._libs.tslib.Timedelta data type.
    '''
    arg = str(days) + ' days'
    return pd.Timedelta(arg)

## Recording the inconsistent instances index 
dropIx = data[data['WaitingDays'] < waiting_days(0)].index
## Dropping these instances from the dataset:
data.drop(dropIx, inplace=True)

data['WaitingDays'] = data.WaitingDays.dt.days  #Extract just the day value from the full "timedelta" object.
data.sample(9)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,ScheduledHour,ScheduleTime,WaitingDays
17364,8.344849e+10,5687092,F,2016-05-11 00:00:00+00:00,2016-05-18 00:00:00+00:00,32,SÃO PEDRO,1,0,0,0,0,0,Yes,14,14:49:04,7
62511,3.952840e+12,5699930,M,2016-05-16 00:00:00+00:00,2016-05-18 00:00:00+00:00,61,DO CABRAL,0,0,0,0,0,0,No,8,08:57:52,2
30641,5.422944e+13,5682885,M,2016-05-11 00:00:00+00:00,2016-05-11 00:00:00+00:00,6,ILHA DE SANTA MARIA,0,0,0,0,0,0,No,6,06:55:18,0
106900,8.674458e+12,5767050,F,2016-06-02 00:00:00+00:00,2016-06-02 00:00:00+00:00,33,JUCUTUQUARA,0,0,0,0,0,0,Yes,16,16:23:16,0
10072,1.236616e+10,5567699,M,2016-04-11 00:00:00+00:00,2016-05-09 00:00:00+00:00,28,MARIA ORTIZ,0,0,0,0,0,0,No,13,13:09:45,28
9316,2.773175e+12,5641260,F,2016-04-29 00:00:00+00:00,2016-05-09 00:00:00+00:00,25,SANTOS DUMONT,0,0,0,0,0,0,Yes,13,13:06:05,10
98399,4.296693e+14,5673486,F,2016-05-09 00:00:00+00:00,2016-06-01 00:00:00+00:00,69,JABOUR,0,0,0,0,0,1,No,9,09:25:41,23
78446,6.327576e+14,5567003,M,2016-04-11 00:00:00+00:00,2016-05-09 00:00:00+00:00,17,BONFIM,0,0,0,0,0,0,No,11,11:27:04,28
26157,1.425167e+14,5484763,M,2016-03-17 00:00:00+00:00,2016-05-18 00:00:00+00:00,38,JOANA D´ARC,0,0,0,0,0,0,No,14,14:44:19,62


## 새로운 Feature 추가 3
[참고 자료](https://www.kaggle.com/code/ahmedmohameddawoud/why-patients-do-not-show-up-for-the-appointment?scriptVersionId=64751637&cellId=58)

In [439]:
#changing the name of No-Show and encoding it to prevent any misconciption

#changing the name
data.rename(columns = {"No-show": "Show"}, inplace = True)

#encoding it
labels = {"No": 1, "Yes":0}
data["Show"] = data["Show"].map(labels)

#converting it into integer
data["Show"] = data["Show"].astype(int)

#Scholar Alcoholism Impact
scholar_alco_impact = pd.pivot_table(data = data, index = ["Scholarship", "Alcoholism"], values = "Show")
round(scholar_alco_impact * 100, 2)

Show
Scholarship Alcoholism       
0           0           80.16
            1           81.36
1           0           76.49
            1           71.78

In [440]:
data["ScholarAndAlcohol"] = data['Scholarship'] & data['Alcoholism']

## 새로운 Feature 추가 4

In [441]:
data["ScholarAndDiabetes"] = data['Scholarship'] & data['Diabetes']

## 새로운 Feature 추가 5
[참고 자료](https://www.kaggle.com/code/somrikbanerjee/predicting-show-up-no-show?scriptVersionId=959321&cellId=48)

In [442]:
def dayToNumber(day):
    if day == 'Monday': 
        return 0
    if day == 'Tuesday': 
        return 1
    if day == 'Wednesday': 
        return 2
    if day == 'Thursday': 
        return 3
    if day == 'Friday': 
        return 4
    if day == 'Saturday': 
        return 5
    if day == 'Sunday': 
        return 6
data["ScheduledDays"] = pd.to_datetime(data['ScheduledDay']).dt.day_name().apply(dayToNumber)
data.sample(10)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Show,ScheduledHour,ScheduleTime,WaitingDays,ScholarAndAlcohol,ScholarAndDiabetes,ScheduledDays
89613,1.883146e+12,5700840,M,2016-05-16 00:00:00+00:00,2016-06-08 00:00:00+00:00,36,ITARARÉ,0,0,0,1,0,1,1,9,09:49:37,23,0,0,0
95225,5.142494e+11,5785984,M,2016-06-08 00:00:00+00:00,2016-06-08 00:00:00+00:00,23,MARIA ORTIZ,0,0,0,0,0,0,1,7,07:54:33,0,0,0,2
63781,1.494334e+13,5736863,M,2016-05-25 00:00:00+00:00,2016-05-30 00:00:00+00:00,12,RESISTÊNCIA,0,0,0,0,0,1,1,8,08:51:08,5,0,0,2
95872,9.262883e+11,5634070,F,2016-04-28 00:00:00+00:00,2016-06-06 00:00:00+00:00,28,MARIA ORTIZ,0,0,0,0,0,1,1,10,10:19:56,39,0,0,3
11261,8.665374e+12,5515917,F,2016-03-28 00:00:00+00:00,2016-05-06 00:00:00+00:00,10,RESISTÊNCIA,0,0,0,0,0,1,1,13,13:46:15,39,0,0,0
38239,3.416681e+13,5745883,F,2016-05-30 00:00:00+00:00,2016-05-30 00:00:00+00:00,6,DO QUADRO,0,0,0,0,0,0,1,12,12:59:22,0,0,0,0
48097,4.311525e+13,5684295,F,2016-05-11 00:00:00+00:00,2016-05-11 00:00:00+00:00,81,GOIABEIRAS,0,1,1,0,0,0,1,8,08:26:37,0,0,0,2
75596,3.711269e+13,5649800,F,2016-05-03 00:00:00+00:00,2016-05-03 00:00:00+00:00,67,NOVA PALESTINA,0,1,0,0,0,0,1,7,07:14:03,0,0,0,1
7165,8.713749e+14,5676565,M,2016-05-09 00:00:00+00:00,2016-05-09 00:00:00+00:00,63,ROMÃO,0,0,0,1,0,0,1,15,15:54:57,0,0,0,0
53261,2.298818e+14,5683680,M,2016-05-11 00:00:00+00:00,2016-05-11 00:00:00+00:00,9,ESTRELINHA,1,0,0,0,0,0,0,7,07:46:00,0,0,0,2


## Test를 위한 Data 준비

In [443]:
data['Gender'] = data['Gender'].replace({'F':0, 'M':1})
data.drop(["PatientId","AppointmentID", "ScheduledDay", "AppointmentDay", "Neighbourhood", "ScheduleTime", "Diabetes", "Alcoholism"],axis=1,inplace=True)
data.isnull().sum()

Gender                0
Age                   0
Scholarship           0
Hipertension          0
Handcap               0
SMS_received          0
Show                  0
ScheduledHour         0
WaitingDays           0
ScholarAndAlcohol     0
ScholarAndDiabetes    0
ScheduledDays         0
dtype: int64

## Model Test

In [444]:
y_train = data['Show']
X_train = data.drop('Show', axis=1)

X_train.head()

,Gender,Age,Scholarship,Hipertension,Handcap,SMS_received,ScheduledHour,WaitingDays,ScholarAndAlcohol,ScholarAndDiabetes,ScheduledDays
0,0,62,0,1,0,0,18,0,0,0,4
1,1,56,0,0,0,0,16,0,0,0,4
2,0,62,0,0,0,0,16,0,0,0,4
3,0,8,0,0,0,0,17,0,0,0,4
4,0,56,0,1,0,0,16,0,0,0,4


In [445]:
from sklearn.model_selection import train_test_split
#Importing the auxiliar and preprocessing librarys 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.metrics import accuracy_score

#Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

In [446]:
#knn, decision tree, random forest, NB, SVM
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.25)

clfs = []
seed = 3

clfs.append(("KNN", 
             Pipeline([("Scaler", StandardScaler()),
                       ("KNN", KNeighborsClassifier())]))) 

clfs.append(("DecisionTreeClassifier", 
             Pipeline([("Scaler", StandardScaler()),
                       ("DecisionTrees", DecisionTreeClassifier())]))) 

clfs.append(("RandomForestClassifier", 
             Pipeline([("Scaler", StandardScaler()),
                       ("RandomForest", RandomForestClassifier())])))

clfs.append(("NB",
             Pipeline([("Scaler", StandardScaler()),
                       ("NB", GaussianNB())]))) 

clfs.append(("LinearSVC",
             Pipeline([("Scaler", StandardScaler()),
                       ("LinearSVC", LinearSVC(C=0.1))]))) 


scoring = 'accuracy'
n_folds = 5

results, names  = [], [] 

for name, model  in clfs:
    kfold = KFold(n_splits=n_folds, shuffle = True, random_state=seed)
    cv_results = cross_val_score(model, X_train, y_train, 
                                 cv=kfold, scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(),  
                               cv_results.std())
    print(msg)

KNN: 0.768706 (+/- 0.001135)
DecisionTreeClassifier: 0.716190 (+/- 0.002649)
RandomForestClassifier: 0.767065 (+/- 0.001451)
NB: 0.774726 (+/- 0.002427)
LinearSVC: 0.795331 (+/- 0.002276)
